In [9]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [10]:
df_gu=pd.read_csv("fire_gu_col.csv",encoding="cp949")
df_gu=df_gu.drop(df_gu.columns[[0,1]],axis=1) #구 별 화재발생 횟수

df_jb=pd.read_csv("fire_jangbi.csv",encoding="cp949") #소방서 별 장비 보유 대수
df_jb=df_jb.replace('-',0)

df_location_119=pd.read_csv("전국 119안전센터 현황(2018.11.28).csv",encoding="cp949")
df_location_119=df_location_119[df_location_119['시도본부']=='서울소방재난본부']
df_location_119['주소']=df_location_119['주소'].apply(lambda e: e.split(' ')[1])
df_location_119=df_location_119.drop(['순번','시도본부','소방서','전화번호'],axis=1)
df_location_119['119안전센터명']=df_location_119['119안전센터명']+'119안전센터'
df_location_119.rename(columns={"119안전센터명":"이름"}, inplace = True)

df_location_fire=pd.read_csv("전국 소방서 현황(2018.11.28).csv",encoding="cp949")
df_location_fire=df_location_fire[df_location_fire['본부명']=='서울소방재난본부']
df_location_fire['주소']=df_location_fire['주소'].apply(lambda e: e.split(' ')[1])
df_location_fire=df_location_fire.drop(['순번','본부명','전화번호'],axis=1)
df_location_fire.rename(columns={"소방서":"이름"}, inplace = True)

df_concat = pd.concat([df_location_119,df_location_fire],ignore_index=True) #소방서, 119 합치기

df1=pd.read_csv("df_firesafe.csv",encoding="cp949") #믄 졍 둘이서 합친거
df1=df1.drop(['Unnamed: 0'],axis=1)

df = pd.merge(df1, df_concat ,how='left', on='이름') #둘이서 합친거에 구 추가

df_people=pd.read_csv("서울시 소방 공무원 (정원) 통계.csv",encoding="cp949")
df_people=df_people.replace('-',0)

## 구별 화재 발생 통계값 대비 소방차 수

In [11]:
df_jb=df_jb[['소방서','합계']].iloc[1:] #소방시설 수
df_jb.rename(columns={"소방서":"이름"}, inplace = True)
df_jb_plus = pd.merge(df_jb, df ,how='inner', on='이름') #둘이서 합친거에 구 추가
df_jb_plus.rename(columns={"주소":"자치구"}, inplace = True)
df_gu=df_gu[['자치구','발생_합계']] #화재발생 통계값
df_jb_plus = pd.merge(df_jb_plus, df_gu ,how='left', on='자치구') #화재발생합계추가
df_jb_plus.rename(columns={"합계":"장비보유수"}, inplace = True)
df_jb_plus.rename(columns={"발생_합계":"화재발생횟수"}, inplace = True)
#금천구 화재발생횟수 구로소방서에 추가
df_jb_plus['화재발생횟수'].iloc[14]=int(df_jb_plus[df_jb_plus['이름']=='구로소방서']['화재발생횟수'])+int(df_gu[df_gu['자치구']=='금천구']['발생_합계'])


df_jb_plus

c:\users\kim seokyeung\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,이름,장비보유수,위도,경도,자치구,화재발생횟수
0,종로소방서,44,37.599100,126.986175,종로구,254
1,중부소방서,40,37.557945,126.994195,중구,275
2,광진소방서,30,37.548145,127.061733,광진구,203
3,용산소방서,36,37.531106,126.980942,용산구,211
4,동대문소방서,40,37.583794,127.050688,동대문구,203
5,영등포소방서,40,37.520641,126.913924,영등포구,258
6,성북소방서,36,37.606989,127.023244,성북구,215
7,은평소방서,37,37.617611,126.922713,은평구,214
8,강남소방서,50,37.495981,127.066409,강남구,436
9,서초소방서,42,37.476953,127.037800,서초구,242


In [12]:
test1=df_jb_plus['화재발생횟수']/df_jb_plus['장비보유수']
test1=pd.DataFrame(test1)
test1['자치구']=df_jb_plus['자치구']
test1.rename(columns={0:"화재수/장비수"}, inplace = True)
tmp=test1.iloc[14]
tmp['자치구']='금천구'
test1=test1.append(tmp,ignore_index=True)
test1 #클수록 화재대비해서 장비보유가 적은 것

c:\users\kim seokyeung\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,화재수/장비수,자치구
0,5.772727,종로구
1,6.875000,중구
2,6.766667,광진구
3,5.861111,용산구
4,5.075000,동대문구
5,6.450000,영등포구
6,5.972222,성북구
7,5.783784,은평구
8,8.720000,강남구
9,5.761905,서초구


In [14]:
test1.to_csv("test1.csv",encoding='ms949')

## 구별 화재 발생 통계값 대비 소방관 수

In [232]:
df_people_plus=df_people[['구분','소방장','소방교','소방사']]
df_people_plus.rename(columns={"구분":"이름"}, inplace = True)
df_people_plus = pd.merge(df_people_plus, df ,how='inner', on='이름') #구 추가
df_people_plus['소방인력']=df_people_plus['소방장'].astype(int)+df_people_plus['소방교'].astype(int)+df_people_plus['소방사'].astype(int)
df_people_plus=df_people_plus.drop(['소방장','소방교','소방사'],axis=1)
df_people_plus.rename(columns={"주소":"자치구"}, inplace = True)
df_people_plus = pd.merge(df_people_plus, df_gu ,how='left', on='자치구') #화재발생합계추가
df_people_plus.rename(columns={"발생_합계":"화재발생횟수"}, inplace = True)

In [233]:
df_people_plus

,이름,위도,경도,자치구,소방인력,화재발생횟수
0,종로소방서,37.599100,126.986175,종로구,257,254
1,중부소방서,37.557945,126.994195,중구,239,275
2,광진소방서,37.548145,127.061733,광진구,177,203
3,용산소방서,37.531106,126.980942,용산구,218,211
4,동대문소방서,37.583794,127.050688,동대문구,238,203
5,중랑소방서,37.595375,127.093966,중랑구,211,254
6,영등포소방서,37.520641,126.913924,영등포구,248,258
7,성북소방서,37.606989,127.023244,성북구,210,215
8,도봉소방서,37.665859,127.031795,도봉구,187,237
9,노원소방서,37.655260,127.077120,노원구,240,229


In [234]:
test2=df_people_plus['화재발생횟수']/df_people_plus['소방인력']
test2=pd.DataFrame(test2)
test2['자치구']=df_people_plus['자치구']
test2.rename(columns={0:"화재수/소방인력수"}, inplace = True)
tmp=test2.iloc[16]
tmp['자치구']='금천구'
test2=test2.append(tmp,ignore_index=True)
test2 #클수록 화재대비해서 소방인력수가 적은 것

c:\users\kim seokyeung\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,화재수/소방인력수,자치구
0,0.988327,종로구
1,1.150628,중구
2,1.146893,광진구
3,0.967890,용산구
4,0.852941,동대문구
5,1.203791,중랑구
6,1.040323,영등포구
7,1.023810,성북구
8,1.267380,도봉구
9,0.954167,노원구


In [241]:
test2.to_csv("test2.csv",encoding='cp949')

## 구별 화재 발생 통계값 대비 소방시설 수

In [181]:
df_gu_plus=pd.DataFrame(df.groupby('주소')['이름'].count().index)
df_gu_plus.rename(columns={"주소":"자치구"}, inplace = True)
df_gu_plus['소방시설수']=df.groupby('주소')['이름'].count().values
df_gu_plus = pd.merge(df_gu, df_gu_plus ,how='inner', on='자치구') #화재발생수랑 구별 소방시설수 합침
df_gu_plus.rename(columns={"발생_합계":"화재발생횟수"}, inplace = True)
df_gu_plus

,자치구,화재발생횟수,소방시설수
0,종로구,254,7
1,중구,275,6
2,용산구,211,6
3,성동구,212,5
4,광진구,203,4
5,동대문구,203,6
6,중랑구,254,5
7,성북구,215,5
8,강북구,203,5
9,도봉구,237,5


In [183]:
test3=df_gu_plus['화재발생횟수']/df_gu_plus['소방시설수']
test3=pd.DataFrame(test3)
test3['자치구']=df_gu_plus['자치구']
test3.rename(columns={0:"화재수/소방시설수"}, inplace = True)
test3 #클수록 화재대비해서 소방시설이 적은 것

,화재수/소방시설수,자치구
0,36.285714,종로구
1,45.833333,중구
2,35.166667,용산구
3,42.400000,성동구
4,50.750000,광진구
5,33.833333,동대문구
6,50.800000,중랑구
7,43.000000,성북구
8,40.600000,강북구
9,47.400000,도봉구
